## Load MNIST Data

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

## Start Tensorflow InteractiveSession

In [ ]:
import tensorflow as tf
sess = tf.InteractiveSession()

Placeholders

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

Variables

In [ ]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [ ]:
sess.run(tf.global_variables_initializer())

## Predicted Class and Loss Function

In [ ]:
y = tf.matmul(x, W) + b
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

## Train the Model

In [ ]:
type(mnist)

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

for _ in range(1000):
        batch = mnist.train.next_batch(128)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

## Evaluate the Model

In [ ]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

# Multilayer Convolutional Network

## Weight Initialization

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [ ]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                         strides=[1, 2, 2, 1], padding="SAME")

First Convolution Layer

In [ ]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

Second Convolution Layer

In [ ]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

Densely Connected Layer

In [ ]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

Dropout

In [ ]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

Readout Layer

In [ ]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

### Train and Evaluate Model

In [ ]:
%%time
# training step
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Evaluation metric
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Train
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        batch = mnist.train.next_batch(64)
        if i % 100 == 0 :
            train_accuracy = accuracy.eval(feed_dict={
                x: batch[0],
                y_: batch[1],
                keep_prob: 1.0
            })
            print("step %d, training accuracy %g" % (i, train_accuracy))
        train_step.run(feed_dict={
            x: batch[0],
            y_: batch[1],
            keep_prob: 0.5
        })
    # Evaluate
    print("test accuracy %g" % accuracy.eval(feed_dict={
        x: mnist.test.images,
        y_: mnist.test.labels,
        keep_prob: 1.0
    }))

### Random Hierarchical Embedding

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomTreesEmbedding
class RandomTreeEncoder:
    def __init__(self, num_trees=10):
        self.num_trees = num_trees
        self.encoders = []
        
    def build_encoders(self, training_set, labels=None):
        self.encoders = []
        for i in range(self.num_trees):
            encoder = RandomTreesEmbedding(
                n_estimators=1,
                max_depth=None#,
#                 min_weight_fraction_leaf=0.00005
#                 min_impurity_decrease=0.01
            )
            encoder.fit(training_set)
            self.encoders.append(encoder)
        if type(labels) is np.ndarray:
            for label in np.unique(labels):
                for i in range(self.num_trees):
                    # the label
                    label_set = training_set[labels==label]
                    encoder = RandomTreesEmbedding(
                        n_estimators=1,
                        max_depth=None
                    )
                    encoder.fit(label_set)
                    self.encoders.append(encoder)
                    # other labels
                    label_set = training_set[labels!=label]
                    encoder = RandomTreesEmbedding(
                        n_estimators=1,
                        max_depth=None
                    )
                    encoder.fit(label_set)
                    self.encoders.append(encoder)

    def encode_features(self, features):
        """
        Encodes a dataframe of features using the set of encoders
        trained on the training set.
        """
        encodings = pd.DataFrame()
        for i in range(len(self.encoders)):
            encoder = self.encoders[i]
            encoding = encoder.transform(features)
            encodings[str(i)] = encoding.nonzero()[1]
        return encodings

In [ ]:
import numpy as np
labels = np.unique(Y)
# for label in labels:
#     print(np.shape(X[Y==label]))

In [ ]:
type(None)

In [ ]:
X = mnist.train.images
Y = np.argmax(mnist.train.labels, 1)
x_prediction = mnist.test.images
y_test = np.argmax(mnist.test.labels, 1)

In [ ]:
len(rte.encoders)

In [ ]:
%%time
rte = RandomTreeEncoder(30)
rte.build_encoders(X, Y)
X_encodings = rte.encode_features(X)
Pred_encodings = rte.encode_features(x_prediction)

In [ ]:
# X_encodings.max()

In [ ]:
X_encodings.head()

In [ ]:
%%time
# train-test split evaluation of xgboost model
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

model = XGBClassifier(max_depth=20, objective="multi:softmax")
kfold = KFold(n_splits=3, random_state=7)
results = cross_val_score(model, X_encodings, Y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
# 96.1% 700 features min_impurity decrease=0.0
# Accuracy: 94.85% (0.03%) 100 features min_impurity decrease=0.0
# Wall time: 4min 17s
# Accuracy: 89.61% (0.14%)
# Wall time: 2min 3s
# Accuracy: 95.17% (0.19%) 100 features min_i_d=0.0 & softmax
# Wall time: 4min 12s
# Accuracy: 96.15% (0.03%) 700 features & softmax
# Wall time: 25min 35s
# Accuracy: 96.03% (0.04%) 700 features & softmax & max_depth 20 instead of 10
# Wall time: 36min 19s
# Accuracy: 95.47% (0.13%) 20 + 200
# Wall time: 9min 34s

In [ ]:
%%time
model = XGBClassifier(max_depth=20, objective="multi:softmax")
# compute predictions
model.fit(X_encodings, Y)
# y_prediction = model.predict_proba(Pred_encodings)
y_prediction = model.predict(Pred_encodings)

In [ ]:
model

In [ ]:
y_test

In [ ]:
y_prediction

In [ ]:
accuracy_score(y_test, y_prediction)
# 0.95540000000000003 with 100 trees
# 0.96330000000000005 with 200 trees
# 0.96830000000000005 with 700 trees
# 0.90659999999999996 with 100 trees and min_weight_fraction_leaf=0.1
# 0.95689999999999997 with 100 trees, softmax and min_weight_fraction_leaf=0.00005
# 0.96140000000000003 with 100 trees & softmax
# 0.96689999999999998 with 700 trees & softmax
# 0.96450000000000002 with 700 trees & softmax & max_depth 20
# 0.96299999999999997 with 20 + 200 trees & max_depth 20
# 0.96679999999999999 with 100 + 1000 trees & max_depth 20
# 0.96479999999999999 with 20 + 400 trees & max_depth 20
# 0.9647              with 20 + 400 trees & max_depth 10
# 0.9446              with 20 + 400 trees & max_depth 3
# 0.96919999999999995 with 100 + 2000 trees & max_depth 20

In [ ]:
top_features = np.argsort(model.feature_importances_)[::-1]
print("Top features: {}".format(top_features))
print("Top Scores: {}".format(model.feature_importances_[top_features]))

### Bin encodings

In [ ]:
def bin_encodings(encodings):
    return (encodings / 100).round()

In [ ]:
model.fit(bin_encodings(X_encodings), Y)

In [ ]:
y_prediction = model.predict(bin_encodings(Pred_encodings))

In [ ]:
top_features = np.argsort(model.feature_importances_)[::-1]
print("Top features: {}".format(top_features))
print("Top Scores: {}".format(model.feature_importances_[top_features]))
# Top features: [2 1 0 3 6 4 5 8 9 7] <= should be like in almost increasing order
# since we feed top 10 features in decreasing order of importances
# Top Scores: [ 0.10375565  0.10359149  0.10310668  0.10089635  0.09991525  0.09964803
#   0.09869365  0.09841879  0.09672765  0.09524646]

In [ ]:
accuracy_score(y_test, y_prediction)
# 0.95509999999999995 with 100 trees
# 0.89259999999999995 with random 10 trees
# 0.90459999999999996 with best 10 trees given by feature importances
# 0.96789999999999998 with 700 trees and bin encodings

### Top Columns

In [ ]:
columns = [str(i) for i in range(10)]
columns

In [ ]:
columns = [str(c) for c in top_features[:10]]
columns

In [ ]:
model.fit(bin_encodings(X_encodings[columns]), Y)

In [ ]:
y_prediction = model.predict(bin_encodings(Pred_encodings[columns]))

In [ ]:
top_features = np.argsort(model.feature_importances_)[::-1]
print("Top features: {}".format(top_features))
print("Top Scores: {}".format(model.feature_importances_[top_features]))
# Top features: [2 1 0 3 6 4 5 8 9 7] <= should be like in almost increasing order
# since we feed top 10 features in decreasing order of importances
# Top Scores: [ 0.10375565  0.10359149  0.10310668  0.10089635  0.09991525  0.09964803
#   0.09869365  0.09841879  0.09672765  0.09524646]

In [ ]:
accuracy_score(y_test, y_prediction)
# 0.95509999999999995 with 100 trees
# 0.89259999999999995 with random 10 trees
# 0.90459999999999996 with best 10 trees given by feature importances

### XGBOOST with no Embeddings

In [ ]:
%%time
model = XGBClassifier(max_depth=5, objective="multi:softmax")
model.fit(X, Y)
y_prediction = model.predict(x_prediction)
# Wall time: 6min 57s

In [ ]:
accuracy_score(y_test, y_prediction)
#  0.97109999999999996
#  0.97160000000000002 with 20 max_depth
#  0.9637 with 5 max_depth